## Overview

This inbound_network sample application walk you through how to enable inbound networking port on Panorama devices, and how to run a simple HTTP server within a Panorama application. In order to focus on the inbound networking topic, this sample application doesn't use any cameras and any models.

#### How this application works

1. Configure inbound networking port information and port mapping between host and container in JSON files, before deployment.
1. In the application, start a thread to serve HTTP requests, and react to HTTP GET requests to some predefined paths. ( "/py_object_stat", "/py_threads" ).
1. Confirm you can see the expected contents by opening the URLs with your PC browser.

## Prerequisites

Before you start processing this notebook, some prerequisites need to be completed.

* Set up your AWS Panorama Appliance - [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-setup.html)
* Install "panorama-cli" tool [middle click to open document](https://docs.aws.amazon.com/panorama/latest/dev/gettingstarted-deploy.html#gettingstarted-deploy-prerequisites)

<div class="alert alert-block alert-warning"><b>Security Warning:</b> Turning on a device port may create a security vulnerability. This sample opens a port to run simple HTTP server just to explain the inbound networking feature. When you open a port in your own application, please be careful not to cause leaking sensitive information, or breaking your application's behavior.</div>


## Import libraries and define configurations

First step is to import all libraries needed.

In [ ]:
import sys
import os
import time
import json
import uuid

import boto3

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

You need to specify some information specific to your environment.

In [ ]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region_name = boto3.session.Session().region_name

print( "account_id :", account_id )
print( "region_name :", region_name )

# Following configurations are required when you use real hardware, 
# thus can be any dummy strings when you use only Test Utility.
device_id = input("Device Id (format : device-*)").strip()

## Import application

With "panorama-cli import-application" command, replacing placeholder information in application files. This step essentially replace placeholder ("123456789012") with your aws account id.

In [ ]:
!cd ./inbound_network_app/ && panorama-cli import-application

## Prepare business logic

#### Preview python source code
Next step is to build a business logic container. This application's business logic consists of single python source code. Let's preview it.

In [ ]:
panorama_test_utility.preview_text_file( f"./inbound_network_app/packages/{account_id}-inbound_network_code-1.0/src/app.py" )

The `Application` class just creates and start a `IntrospectionHttpServerThread` instance, and run main loop without using cameras, models, and HDMI output. This application focuses on the explanation of inbound networking feature.

---------
``` python
# Start a http server thread
self.http_server_thread = IntrospectionHttpServerThread()
self.http_server_thread.start()
```
---------


In the `IntrospectionHttpServerThread.run()`, it creates a `TCPServer` with `IntrospectionHttpRequestHandler` with port number `8080`.

---------
``` python
PORT = 8080
with socketserver.TCPServer(("", PORT), IntrospectionHttpRequestHandler) as httpd:
    :
```
---------


`IntrospectionHttpRequestHandler.do_GET()` handles HTTP GET requests to "/py_object_stat" and "/py_threads". For other paths it returns 404 error.

---------
``` python
if self.path=="/py_object_stat":

    self.send_response(200)
    self.send_header('Content-Type', 'text/plain')
    self.end_headers()

    message = get_py_object_stat()
    message = message.encode("utf-8")

    self.wfile.write(message)
        :
```
---------

`get_py_object_stat()` is a helper functions to return numbers of python objects for each type in a string.
`get_py_threads()` is a helper functions to return call-stacks of all python threads in a string.
These functions are used to generate response body of HTTP requests.


#### Test run the business logic with test-utility

Let's run the application with Test Utility **Run** command. In this sample, we don't use any model, so model compilation step is not needed.

Now the application is running HTTP server at port 8080. Please open a new tab on your browser, and open URLs. `http://{ip address}:8080/py_object_stat` or `http://{ip address}:8080/py_threads`, and make sure you can see numbers of python objects and call-stacks.

**Note:** depending on your environment such as firewall setting, port 8080 may not be accessible from your browser. In that case please move on to next steps without confirming the result here.

<div class="alert alert-block alert-warning"><b>How to stop:</b> This application runs infinitely, and doesn't end automatically. From the menu bar, please select "Kernel" > "Interrupt Kernel" to abort the application.</div>

In [ ]:
# Run the application with test-utility.

%run ../common/test_utility/panorama_test_utility_run.py \
--app-name inbound_network_app \
--code-package-name inbound_network_code \
--py-file ./inbound_network_app/packages/{account_id}-inbound_network_code-1.0/src/app.py

#### Build application logic container

With "panorama-cli build-container" command, building a container image, and add it into the "inbound_network_code" package.

This step takes long time (5~10 mins), and because it is using %%capture magic command, you don't see progress during the process. Please wait.

In [ ]:
%%capture captured_output
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./inbound_network_app && panorama-cli build-container \
    --container-asset-name code \
    --package-path packages/{account_id}-inbound_network_code-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

#### Define inbound networking port in the code package JSON file

In order to enable inbound networking port, you need to include necessary information in 2 places.
1. package.json of the code package.
2. override manifest file.

Let's make sure `package.json` of the code package contains the port number we want to use. In this sample, the package.json file already contains "nodePackage" > "interfaces" > "network" > "inboundPorts" element.

``` json
"network": {
    "inboundPorts": [
        {
            "port": 8080,
            "description": "http"
        }
    ]
}
```


In [ ]:
panorama_test_utility.preview_text_file( f"./inbound_network_app/packages/{account_id}-inbound_network_code-1.0/package.json" )

## Package application (upload locally prepared packages onto Cloud)

Now you have prepared code package locally, and confirmed that package.json contains "inboundPorts" element. Let's upload the package to the cloud with "panorama-cli package-application" command.

In [ ]:
!cd ./inbound_network_app && panorama-cli package-application

## Deploy application to the device programatically

Once you uploaded the package to the cloud, you can create an application instance on your device. You need to specify a manifest file and an override manifest file.

As explained above, you need to make sure that override manifest file contains necessary information. In this sample, the override.json file already contains "nodeGraphOverrides" > "networkRoutingRules" element, and it contains a mapping between the container port (8080) and host port (8081). This means application's business logic container uses port number 8080, and it is mapped to port number 8081 of the device.


**Note:** "networkRoutingRules" has to be defined in the **override** manifest file, not in the main manifest file. "networkRoutingRules" in the main manifest file is used only to provide default values when you deploy your application via the Management Console UI.

``` json
"networkRoutingRules":[
    {
        "node": "code_node",
        "containerPort": 8080,
        "hostPort": 8081
    }
]
```


In [ ]:
panorama_test_utility.preview_text_file( "./inbound_network_app/graphs/inbound_network_app/graph.json" )

In [ ]:
panorama_test_utility.preview_text_file( "./inbound_network_app/graphs/inbound_network_app/override.json" )

#### Deploy the app using the manifest file and override manifest file

In order to create an application instance, this notebook uses boto3's "panorama" client and its create_application_instance() API. (It is also possible to use "aws panorama create-application-instance" command instead.)

In [ ]:
# create a boto3 client to access Panorama service
# FIXME : not specifying region name here, because panorama-cli uses only default region currently.
panorama_client = boto3.client("panorama")

In [ ]:
def deploy_application( application_name, manifest_filename, override_filename ):

    def get_payload_from_json( filename ):
        with open( filename ) as fd:
            
            s = fd.read()
            
            # validating JSON format and making it compact, by loading and dumping, 
            payload = json.dumps(json.loads(s))
            
            return payload

    manifest_payload = get_payload_from_json( manifest_filename )
    
    params = {
        "Name" : application_name,
        "DefaultRuntimeContextDevice" : device_id,
        "ManifestPayload" : {"PayloadData":manifest_payload},
    }
    
    if override_filename:
        override_payload = get_payload_from_json( override_filename )
        params["ManifestOverridesPayload"] = {"PayloadData":override_payload}
    
    response = panorama_client.create_application_instance( ** params )
        
    return response

In [ ]:
application_name = "inbound_network_notebook_" + str(uuid.uuid4())[:8]

response = deploy_application(
    application_name = application_name,
    manifest_filename = "./inbound_network_app/graphs/inbound_network_app/graph.json",
    override_filename = "./inbound_network_app/graphs/inbound_network_app/override.json",
)

application_instance_id = response["ApplicationInstanceId"]

response

In [ ]:
application_name

#### Wait for deployment completion

Application instance creation has been triggered. This notebook checks the progress by calling describe_application_instance() API periodically. Please confirm that you see "DEPLOYMENT_SUCCEEDED" status at the end.

In [ ]:
def wait_deployment( application_instance_id ):
    
    progress_dots = panorama_test_utility.ProgressDots()    
    while True:
        app = panorama_client.describe_application_instance( ApplicationInstanceId = application_instance_id )
        progress_dots.update_status( "%s (%s)" % (app["Status"], app["StatusDescription"]) )
        if app["Status"] not in ( "DEPLOYMENT_PENDING", "DEPLOYMENT_REQUESTED", "DEPLOYMENT_IN_PROGRESS" ):
            break
        time.sleep(60)

wait_deployment( application_instance_id )

#### Visit CloudWatch Logs to check logs from the application instance

If you saw "DEPLOYMENT_SUCCEEDED" status, the application started to run on your device. Application logs are uploaded to CloudWatch Logs. Let's get the URL of CloudWatch Logs management console. "console_output" is the log stream your Python code's stdout/stderr are redirected to.

In [ ]:
logs_url = panorama_test_utility.get_logs_url( region_name, device_id, application_instance_id )
print( "CloudWatch Logs URL :" )
print( logs_url )

#### Open HTTP URLs on the Panorama device

Let's open a new tab on your browser, and open following HTTP URLs.

* `http://{ip-address-of-the-device}:8081/py_object_stat`
* `http://{ip-address-of-the-device}:8081/py_threads`

You can check the ip address of your device either on the Management Console Device Setting screen, or by executing following cell.

In [ ]:
response = panorama_client.describe_device( DeviceId = device_id )
response["CurrentNetworkingStatus"]

<img src="images/screenshot1.png" alt="screenshot1" style="width: 600px;"/>
<img src="images/screenshot2.png" alt="screenshot2" style="width: 600px;"/>


#### Remove the application instance from the device

Once you confirm that this application is running as expected, we can remove it from the device. Please enter Y or N in the text box.

In [ ]:
answer = input("Remove the application? [yN]")
if answer.lower()=="y":
    panorama_test_utility.remove_application( device_id, application_instance_id )